## 1. Test Random Environment

In [88]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import pandas as pd

In [103]:
malData=pd.read_csv('datasets/malicious_data_generated.csv')
malData=malData.to_numpy()

malDataTest=np.copy(malData[0,:])
malDataTest=malDataTest.reshape(1,-1)
print(malData[random.randint(0,499),:])
print(malDataTest.shape)
malDataTest[0][1]=malDataTest[0][1]+1
print(malDataTest)

[-36. -20. -94.  83. -41.  21.  66. -36.  12.  86.]
(1, 10)
[[-60. -34. -35. -69.  73.  61. -62.  82. -44.  59.]]


In [90]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, decrease, increse, none
        self.action_space = Discrete(20)
        # max-min array
        self.observation_space = Box(np.asarray([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]),np.asarray([100, 100, 100, 100, 100, 100, 100, 100, 100, 100]))
        # Set start 
        self.state = malData[random.randint(0,499),:]
        # Set time 
        self.length = 60
        
    def step(self, action):
        # Apply action
        if(action<10):
            if(action==0):
                self.state[0][0]+=self.state[0][0]+1
            elif(action==1):
                 self.state[0][1]+=self.state[0][1]+1
            elif(action==2):
                 self.state[0][2]+=self.state[0][2]+1
            elif(action==3):
                 self.state[0][3]+=self.state[0][3]+1
            elif(action==4):
                 self.state[0][4]+=self.state[0][4]+1
            elif(action==5):
                 self.state[0][5]+=self.state[0][5]+1
            elif(action==6):
                 self.state[0][6]+=self.state[0][6]+1
            elif(action==7):
                 self.state[0][7]+=self.state[0][7]+1
            elif(action==8):
                 self.state[0][8]+=self.state[0][8]+1
            else:
                 self.state[0][9]+=self.state[0][9]+1                     
        else:
            if(action==10):
                self.state[0][0]+=self.state[0][0]-1
            elif(action==11):
                 self.state[0][1]+=self.state[0][1]-1
            elif(action==12):
                 self.state[0][2]+=self.state[0][2]-1
            elif(action==13):
                 self.state[0][3]+=self.state[0][3]-1
            elif(action==14):
                 self.state[0][4]+=self.state[0][4]-1
            elif(action==15):
                 self.state[0][5]+=self.state[0][5]-1
            elif(action==16):
                 self.state[0][6]+=self.state[0][6]-1
            elif(action==17):
                 self.state[0][7]+=self.state[0][7]-1
            elif(action==18):
                 self.state[0][8]+=self.state[0][8]-1
            else:
                 self.state[0][9]+=self.state[0][9]-1  
        
        self.length -= 1 
            
        
        # Calculate reward
        rewardMulti=0 
        
        if(self.state[0][0]>=-0.290698 and self.state[0][0]<=-133.441860):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][1]>=0 and self.state[0][1]<=1184):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][2]>=-0.666667 and self.state[0][2]<=10.666667):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][3]>=-0.312383 and self.state[0][3]<=109.259173):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][4]>=0 and self.state[0][4]<=30):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][5]>=-0.322 and self.state[0][5]<=127.488889):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][6]>=-0.282353 and self.state[0][6]<=147.976471):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][7]>=-0.164688 and self.state[0][7]<=715.616633):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][8]>=-0.324081 and self.state[0][8]<=106.407677):
                 rewardMulti=rewardMulti+1
        elif(self.state[0][9]>=-0.750000 and self.state[0][9]<=227.5):
                 rewardMulti=rewardMulti+1
                 
        if(rewardMulti==0):
            reward=-1
        else:
            reward=rewardMulti
                 
        
        # Check if is done
        if self.length <= 0: 
            done = True
        else:
            done = False
        
        # Apply noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset 
        self.state = malData[random.randint(0,499),:]
        # Reset time
        self.length = 60 
        return self.state


In [91]:
env = ShowerEnv()

D:\programas\anaco\envs\jupy\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [92]:
env.observation_space.sample()

array([ 9.9961128e+01, -6.1085159e+01,  8.2159950e+01, -7.3800880e+01,
       -6.3455292e+01,  2.5048004e+01,  5.3916868e-02,  6.8291855e+01,
       -9.9786400e+01,  9.6052071e+01], dtype=float32)

In [93]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

IndexError: invalid index to scalar variable.

## 2. Create a Deep Learning Model with Keras

In [ ]:

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

3

In [ ]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()   
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
#del model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


## 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 5:09 - reward: -1.0000

C:\Users\janus\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\janus\AppData\Roaming\Python\Python38\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 1821/10000 [====>.........................] - ETA: 40s - reward: -0.3487done, took 9.027 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 58.000, steps: 60
Episode 9: reward: 58.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 58.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 58.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 58.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: -60.000, steps: 60
Episode 26: reward: -60.00

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 58.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: -60.000, steps: 60


## 4. Reloading Agent from Memory

In [ ]:
#dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:

#dqn.load_weights('dqn_weights.h5f')

In [ ]:
#_ = dqn.test(env, nb_episodes=5, visualize=True)